# Semantic Segmentation Lab
In this lab, you will build a deep learning network that locates a particular human target within an image.  The premise is that a quadcopter (simulated) is searching for a target, and then will follow the target once found.  It's not enough to simply say the target is present in the image in this case, but rather to know *where* in the image the target is, so that the copter can adjust its direction in order to follow.

Consequently, an image classification network is not enough to solve the problem. Intead, a semantic segmentation network is needed so that the target can be specifically located within the image.

## Data Collection
We have provided you with the dataset for this lab. If you haven't already downloaded the training and validation datasets, you can check out the README for this lab's repo for instructions as well.

In [1]:
import os
import glob
import sys
import tensorflow as tf

from scipy import misc
import numpy as np

from tensorflow.contrib.keras.python import keras
from tensorflow.contrib.keras.python.keras import layers, models

from tensorflow import image

from utils import scoring_utils
from utils.separable_conv2d import SeparableConv2DKeras, BilinearUpSampling2D
from utils import data_iterator

In [2]:
from distutils.version import LooseVersion
import warnings

assert LooseVersion(tf.__version__) >= LooseVersion('1.2'), 'Please use TensorFlow version 1.2 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.3.0
Default GPU Device: /gpu:0


## FCN Layers
In the Classroom, we discussed the different layers that constitute a fully convolutional network. The following code will intoduce you to the functions that you will be using to build out your model.

### Separable Convolutions
The Encoder for your FCN will essentially require separable convolution layers. Below we have implemented two functions - one which you can call upon to build out separable convolutions or regular convolutions. Each with batch normalization and with the ReLU activation function applied to the layers. 

While we recommend the use of separable convolutions thanks to their advantages we covered in the Classroom, some of the helper code we will present for your model will require the use for regular convolutions. But we encourage you to try and experiment with each as well!

The following will help you create the encoder block and the final model for your architecture.

In [3]:
def separable_conv2d_batchnorm(input_layer, filters, strides=1):
    output_layer = SeparableConv2DKeras(filters=filters,kernel_size=3, strides=strides,
                             padding='same', activation='relu')(input_layer)
    
    output_layer = layers.BatchNormalization()(output_layer) 
    return output_layer

def conv2d_batchnorm(input_layer, filters, kernel_size=3, strides=1):
    output_layer = layers.Conv2D(filters=filters, kernel_size=kernel_size, strides=1, 
                      padding='same', activation='relu')(input_layer)
    
    output_layer = layers.BatchNormalization()(output_layer) 
    return output_layer

### Bilinear Upsampling
The following helper function will help implement the bilinear upsampling layer. Currently, upsampling by a factor of 2 is recommended but you can try out different factors as well. You will use this to create the decoder block later!

In [4]:
def bilinear_upsample(input_layer):
    output_layer = BilinearUpSampling2D((2,2))(input_layer)
    return output_layer

### Build the Model
In the following cells, we will cover how to build the model for the task at hand. 

- We will first create an Encoder Block, where you will create a separable convolution layer using an input layer and the size(depth) of the filters as your inputs.
- Next, you will create the Decoder Block, where you will create an upsampling layer using bilinear upsampling, followed by a layer concatentaion, and some separable convolution layers.
- Finally, you will combine the above two and create the model. In this step you will be able to experiment with different number of layers and filter sizes for each to build your model.

Let's cover them individually below.

#### Encoder Block
Below you will create a separable convolution layer using the separable_conv2d_batchnorm() function. The `filters` parameter defines the size or depth of the output layer. For example, 32 or 64. 

In [5]:
def encoder_block(input_layer, filters, strides):
    
    # TODO Create a separable convolution layer using the separable_conv2d_batchnorm() function.
    output_layer = separable_conv2d_batchnorm(input_layer, filters, strides)
    
    return output_layer

#### Decoder Block
The decoder block, as covered in the Classroom, comprises of three steps -

- A bilinear upsampling layer using the upsample_bilinear() function. The current recommended factor for upsampling is set to 2.
- A layer concatenation step. This step is similar to skip connections. You will concatenate the upsampled small_ip_layer and the large_ip_layer.
- Some (one or two) additional separable convolution layers to extract some more spatial information from prior layers.

In [6]:
def decoder_block(small_ip_layer, large_ip_layer, filters):
    # TODO Upsample the small input layer using the bilinear_upsample() function.
    upsampled_layer = bilinear_upsample(small_ip_layer)
    
    # TODO Concatenate the upsampled and large input layers using layers.concatenate
    output_layer = layers.concatenate([upsampled_layer, large_ip_layer])
    
    # TODO Add some number of separable convolution layers
    output_layer = separable_conv2d_batchnorm(output_layer, filters)
    
    return output_layer

#### Model

Now that you have the encoder and decoder blocks ready, you can go ahead and build your model architecture! 

There are three steps to the following:
- Add encoder blocks to build out initial set of layers. This is similar to how you added regular convolutional layers in your CNN lab.
- Add 1x1 Convolution layer using conv2d_batchnorm() function. Remember that 1x1 Convolutions require a kernel and stride of 1.
- Add decoder blocks for upsampling and skip connections.

In [7]:
def fcn_model(inputs, num_classes):
    
    # TODO Add Encoder Blocks. 
    # Remember that with each encoder layer, the depth of your model (the number of filters) increases.
    e0 = encoder_block(inputs, 16, 2)
    e1 = encoder_block(e0, 32, 2)
    e2 = encoder_block(e1, 64, 2)
    e3 = encoder_block(e2, 128, 2)
    
    # TODO Add 1x1 Convolution layer using conv2d_batchnorm().
    bn = conv2d_batchnorm(e3,128,kernel_size=1)
    
    # TODO: Add the same number of Decoder Blocks as the number of Encoder Blocks
    d1 = decoder_block(bn, e2, 128)
    d2 = decoder_block(d1, e1, 64)
    d3 = decoder_block(d2, e0,32 )
    x = decoder_block(d3, inputs, 16)
    
    
    
    # The function returns the output layer of your model. "x" is the final layer obtained from the last decoder_block()
    return layers.Conv2D(num_classes, 3, activation='softmax', padding='same')(x)

### Training
The following cells will utilize the model you created and define an ouput layer based on the input and the number of classes.Following that you will define the hyperparameters to compile and train your model!

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""

image_hw = 128
image_shape = (image_hw, image_hw, 3)
inputs = layers.Input(image_shape)
num_classes = 3

# Call fcn_model()
output_layer = fcn_model(inputs, num_classes)

#### Hyperparameters
Below you can define and tune your hyperparameters

In [9]:
learning_rate = 0.001
batch_size = 128
num_epochs = 25

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
# Define the Keras model and compile it for training
model = models.Model(inputs=inputs, outputs=output_layer)

model.compile(optimizer=keras.optimizers.Adam(learning_rate), loss='categorical_crossentropy')

# Data iterators for loading the training and validation data
train_iter = data_iterator.BatchIteratorSimple(batch_size=batch_size,
                                               data_folder=os.path.join('..', 'data', 'train'),
                                               image_shape=image_shape,
                                               shift_aug=True)

val_iter = data_iterator.BatchIteratorSimple(batch_size=batch_size,
                                             data_folder=os.path.join('..', 'data', 'validation'),
                                             image_shape=image_shape)

model.fit_generator(train_iter,
                    steps_per_epoch = 500, # the number of batches per epoch,
                    epochs = num_epochs, # the number of epochs to train for,
                    validation_data = val_iter, # validation iterator
                    validation_steps = 50, # the number of batches to validate on
                    workers = 2)

Epoch 1/25
500/500 [==============================] - 445s - loss: 0.1855 - val_loss: 0.0721
Epoch 2/25
500/500 [==============================] - 440s - loss: 0.0278 - val_loss: 0.0404
Epoch 3/25
500/500 [==============================] - 441s - loss: 0.0226 - val_loss: 0.0395
Epoch 4/25
500/500 [==============================] - 441s - loss: 0.0203 - val_loss: 0.0399
Epoch 5/25
500/500 [==============================] - 441s - loss: 0.0190 - val_loss: 0.0370
Epoch 6/25
500/500 [==============================] - 441s - loss: 0.0190 - val_loss: 0.0345
Epoch 7/25
500/500 [==============================] - 441s - loss: 0.0198 - val_loss: 0.0343
Epoch 8/25
500/500 [==============================] - 443s - loss: 0.0173 - val_loss: 0.0340
Epoch 9/25
500/500 [==============================] - 441s - loss: 0.0170 - val_loss: 0.0331
Epoch 10/25
500/500 [==============================] - 440s - loss: 0.0158 - val_loss: 0.0343
Epoch 11/25
500/500 [==============================] - 442s - loss: 0

In [11]:
# Save your trained model weights
weight_file_name = 'model_weights_big_network'
model.save_weights(os.path.join('..', 'data', 'weights', weight_file_name))

### Prediction

In [12]:
# If you need to load a model which you previously trained you can uncomment the codeline that calls the following function.
def load_weights(your_model, your_weight_filename):
    model_path = os.path.join('..', 'data', 'weights', your_weight_filename)
    if os.path.exists(model_path):
        model = your_model.load_weights(model_path)
        return model
    else:
        raise ValueError('No weight file found at {}'.format(model_path))

# model = load_weights(model, weight_file_name)

In [13]:
def make_dir_if_not_exist(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [14]:
# NOTE only modify these lines if you have changed where data is being stored(not recommended)
validation_path = os.path.join('..', 'data', 'validation')
file_names = sorted(glob.glob(os.path.join(validation_path, 'images', '*.jpeg')))

In [15]:
experiment_name = 'prediction'# TODO add the name of folder to save these predictions to
output_path = os.path.join('..', 'data', 'runs', experiment_name)
make_dir_if_not_exist(output_path)

In [16]:
for name in file_names:
    image = misc.imread(name)
    if image.shape[0] != image_shape[0]:
         image = misc.imresize(image,image_shape)
    image = data_iterator.preprocess_input(image.astype(np.float32))
    pred = model.predict_on_batch(np.expand_dims(image, 0))
    base_name = os.path.basename(name).split('.')[0]
    base_name = base_name + '_prediction.png'
    misc.imsave(os.path.join(output_path, base_name), np.squeeze((pred * 255).astype(np.uint8)))

### Evaluation
Let's evaluate your model!

In [17]:
scoring_utils.score_run(validation_path, output_path)

number of validation samples intersection over the union evaulated on 1184
average intersection over union for background is 0.9932846495153487
average intersection over union for hero is 0.4130629056037957
average intersection over union for other people is 0.16145285632529788
global average intersection over union is 0.5226001371481475
